Inportamos las Librerias

In [7]:
import numpy as np
import pandas as pd
import json
import joblib
from sklearn.preprocessing import StandardScaler
# modelling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import SGDRegressor
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb


In [54]:
df = pd.read_csv("listings.csv", index_col= "id")


In [55]:
df.drop(columns=['license','last_review'],inplace=True)
df['host_name'].fillna('Unknown',inplace=True)
df['price'].fillna((df['price'].mean()),inplace=True)
df['reviews_per_month'].fillna(0,inplace=True)

In [69]:
data.columns

Index(['name', 'host_id', 'host_name', 'neighbourhood_group', 'neighbourhood',
       'latitude', 'longitude', 'room_type', 'price', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365',
       'number_of_reviews_ltm', 'license'],
      dtype='object')

In [66]:
df.isnull().sum() * 100 / len(data)

name                              0.0
host_id                           0.0
host_name                         0.0
neighbourhood_group               0.0
neighbourhood                     0.0
latitude                          0.0
longitude                         0.0
room_type                         0.0
price                             0.0
minimum_nights                    0.0
number_of_reviews                 0.0
reviews_per_month                 0.0
calculated_host_listings_count    0.0
availability_365                  0.0
number_of_reviews_ltm             0.0
dtype: float64

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34049 entries, 34421962 to 891631866049506737
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   name                            34049 non-null  object 
 1   host_id                         34049 non-null  int64  
 2   host_name                       34049 non-null  object 
 3   neighbourhood_group             34049 non-null  object 
 4   neighbourhood                   34049 non-null  object 
 5   latitude                        34049 non-null  float64
 6   longitude                       34049 non-null  float64
 7   room_type                       34049 non-null  object 
 8   price                           34049 non-null  float64
 9   minimum_nights                  34049 non-null  int64  
 10  number_of_reviews               34049 non-null  int64  
 11  reviews_per_month               34049 non-null  float64
 12  calculated_host_l

In [68]:
df.head()

,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
id,,,,,,,,,,,,,,,
34421962,Waipouli Beach Resort E106,34386367,Kauai Calls!,Kauai,Kapaa-Wailua,22.062280,-159.319180,Entire home/apt,417.000000,1,0,0.00,41,243,0
40707945,DO NOT PUBLISH BEFORE MERGING 14891-HROV,182704096,Aston Waikiki Beach Hotel,Honolulu,Primary Urban Center,21.272438,-157.822556,Private room,290.000000,1,0,0.00,9,365,0
43656551,"Ilikai 1212 Ocean / Lagoon / King Bed, Sofa Bed",347952914,Hawaii Vacation,Honolulu,Primary Urban Center,21.284374,-157.838440,Entire home/apt,199.000000,1,52,1.51,79,145,13
39096262,$900 rooms for rent,299657655,Chelsea,Honolulu,Primary Urban Center,21.340290,-157.878160,Private room,409.804454,365,0,0.00,1,0,0
735005738415289686,"The Point at Poipu, a Hilton Vacation Club, Kauai",24157156,David,Kauai,Koloa-Poipu,21.870674,-159.441671,Private room,409.804454,7,0,0.00,1,353,0


In [74]:
def outliers (df):
    for columnas in df:
        if df[columnas].dtype in ['int64', 'float64']:  

            Q1 = df[columnas].quantile(0.25)
            Q3 = df[columnas].quantile(0.75)
            IQR = Q3 - Q1
            outlierIzq = Q1-(1.5*IQR)
            outlierDer = Q3+(1.5*IQR)
            outliers = df[columnas][(df[columnas] < outlierIzq) | (df[columnas] > outlierDer)]
            print("Variable " + columnas + " has (" + str(len(outliers)) + ") outliers")

    
outliers(df)

Variable host_id has (0) outliers
Variable latitude has (6080) outliers
Variable longitude has (0) outliers
Variable price has (3011) outliers
Variable minimum_nights has (3764) outliers
Variable number_of_reviews has (3699) outliers
Variable reviews_per_month has (2061) outliers
Variable calculated_host_listings_count has (3182) outliers
Variable availability_365 has (0) outliers
Variable number_of_reviews_ltm has (3517) outliers


In [75]:
def reparar_atipicos(df, col):
    Q1 = col.quantile(0.25)
    Q3 = col.quantile(0.75)
    IQR = Q3 - Q1
    
    atipico_inf = Q1 - (1.5 * IQR)
    atipico_sup = Q3 + (1.5 * IQR)
    
    for indice in range(len(df[col.name])): 
        if indice in df.index:  
            elemento = df.loc[indice, col.name] 
            if elemento > atipico_sup: 
                df.loc[indice, col.name] = atipico_sup
            elif elemento < atipico_inf: 
                df.loc[indice, col.name] = atipico_inf
            
    return df  

#repair the outliers of the variable Price
reparar_atipicos(df, df['price'])

,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
id,,,,,,,,,,,,,,,
34421962,Waipouli Beach Resort E106,34386367,Kauai Calls!,Kauai,Kapaa-Wailua,22.062280,-159.319180,Entire home/apt,417.000000,1,0,0.00,41,243,0
40707945,DO NOT PUBLISH BEFORE MERGING 14891-HROV,182704096,Aston Waikiki Beach Hotel,Honolulu,Primary Urban Center,21.272438,-157.822556,Private room,290.000000,1,0,0.00,9,365,0
43656551,"Ilikai 1212 Ocean / Lagoon / King Bed, Sofa Bed",347952914,Hawaii Vacation,Honolulu,Primary Urban Center,21.284374,-157.838440,Entire home/apt,199.000000,1,52,1.51,79,145,13
39096262,$900 rooms for rent,299657655,Chelsea,Honolulu,Primary Urban Center,21.340290,-157.878160,Private room,409.804454,365,0,0.00,1,0,0
735005738415289686,"The Point at Poipu, a Hilton Vacation Club, Kauai",24157156,David,Kauai,Koloa-Poipu,21.870674,-159.441671,Private room,409.804454,7,0,0.00,1,353,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755988499595645037,"Kona Coast Resort 1 Bedroom, full kitchen",457152913,Todd,Hawaii,North Kona,19.582930,-155.968110,Entire home/apt,409.804454,2,3,0.21,85,89,1
756678668207738929,Stunning *Paniolo Greens* 2B#3,340080088,Bianka,Hawaii,South Kohala,19.927620,-155.793990,Entire home/apt,409.804454,2,0,0.00,68,0,0
969798240699256199,Thanksgiving in Kauai - 2bd/2ba,462468097,Shariq,Kauai,Lihue,21.962098,-159.349913,Private room,409.804454,5,0,0.00,1,0,0


ValueError: could not convert string to float: 'Waipouli Beach Resort E106'